In [1]:
import time
from time import localtime, strftime

start = time.time()
t = strftime("%Y_%m_%d_%H_%M_%S", localtime())
print (t)

2017_11_09_19_49_01


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler 
from sklearn.cross_validation import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils

C:\Users\Will\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
# load dataset
train = pd.read_csv(r'C:\Users\Will\Documents\Data Science\Kaggle\Digit Recognizer\train.csv')
test = pd.read_csv(r'C:\Users\Will\Documents\Data Science\Kaggle\Digit Recognizer\test.csv')

In [4]:
data = train.copy()
test_data = test.copy()

In [5]:
print('Sanity Check -- before:')
print(data.shape[1])
print(len(data.label.unique()))

Sanity Check -- before:
785
10


In [6]:
#prepare data
X = data.drop('label', axis = 1)
y = data['label']

print('Sanity Check -- after:')
dims = X.shape[1]
print(dims, 'dims')

nb_classes = len(y.unique())
print(nb_classes, 'classes')

Sanity Check -- after:
784 dims
10 classes


In [7]:
#standardize the feature training data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

#convert target vector to categorical matrix
y = np_utils.to_categorical(y) 

### Create model

In [8]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=dims, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(nb_classes, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Add early stopping to avoid overfitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

fBestModel = 'best_model.h5' 
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1) 
best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=True, validation_split=0.15, 
          callbacks=[best_model, early_stop]) 

Train on 30345 samples, validate on 5355 samples
Epoch 1/150
30345/30345 [==============================] - 91s - loss: 0.3307 - acc: 0.8708 - val_loss: 0.2089 - val_acc: 0.9222
Epoch 2/150
20270/30345 [===================>..........] - ETA: 4s - loss: 0.1783 - acc: 0.9341

In [ ]:
model.evaluate(X_test,y_test, batch_size=10)

In [46]:
#standardize the feature training data
scaler = StandardScaler()
scaler.fit(test_data)
test_data = scaler.transform(test_data)

In [47]:
results = model.predict_classes(test_data,batch_size=10)

27010/28000 [===========================>..] - ETA: 0s

In [48]:
t = strftime("%Y_%m_%d_%H_%M_%S", localtime())
results = pd.DataFrame(results)
results.index.rename('ImageId', inplace=True)
results.columns = ['Label']
filename = 'results_'+t+'.csv'
results.to_csv(filename)